In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from io import BytesIO
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import sqlite3

In [2]:
def get_market_short(market, end_date=None ,start_date=None ):
    market_dict={'kospi':'1','kospi_prod':'2','kosdaq':'3'}
    market_code=market_dict[market]
    if start_date is not None and end_date is None:
        end_date = (datetime.strptime(start_date, '%Y%m%d')+relativedelta(months=3)-relativedelta(days=1)).strftime('%Y%m%d')
    if start_date is None and end_date is None:
        end_date =datetime.today().strftime('%Y%m%d')
        start_date =(datetime.strptime(end_date, '%Y%m%d')-relativedelta(months=3)).strftime('%Y%m%d')
    if start_date is None and end_date is not None:
        start_date = (datetime.strptime(end_date, '%Y%m%d')-relativedelta(months=3)).strftime('%Y%m%d')
    gen_otp_url='http://short.krx.co.kr/contents/COM/GenerateOTP.jspx'
    gen_otp_data={
        'name':'fileDown',
        'filetype':'xls',
        'url':'SRT/02/02020100/srt02020100',
        'mkt_tp_cd':market_code,
        'isu_cdnm':'전체',
        'isu_cd': '',
        'isu_nm': '',
        'isu_srt_cd':'', 
        'strt_dd':start_date,
        'end_dd':end_date,
        'pagePath':'/contents/SRT/02/02020100/SRT02020100.jsp',
    }
    r = requests.post(gen_otp_url, gen_otp_data)
    code = r.content

    down_url = 'http://file.krx.co.kr/download.jspx'
    down_data = {
        'code':code,
    }

    r=requests.post(down_url,down_data)
    r.encoding = "utf-8-sig"
    df = pd.read_excel(BytesIO(r.content), header=0, thousands=',')
    df.columns = ['date','code','name','short_volumes','total_volumes',
        'weight(%)', 'short_values','total_values','weight(%)']
    df['code'] = list(map(lambda v : v[3:9], df['code']))
    
    return df

In [3]:
today=datetime.strftime(datetime.today().date(), '%Y%m%d')
fromdate='20180901'
df = get_market_short('kospi',None,fromdate)

df.tail()

,date,code,name,short_volumes,total_volumes,weight(%),short_values,total_values,weight(%)
20984,2018/09/03,069260,휴켐스,808,71629,1.13,20359900,1800778050,1.13
20985,2018/09/03,000540,흥국화재,1310,35339,3.71,7833890,211785870,3.70
20986,2018/09/03,000542,흥국화재2우B,0,2187,0.00,0,64637350,0.00
20987,2018/09/03,000541,흥국화재우,0,2629,0.00,0,14880480,0.00
20988,2018/09/03,003280,흥아해운,2335,488680,0.48,1276223,266827821,0.48


In [4]:
def update():
    con = sqlite3.connect('D:\croling/short_data.db')
    cur = con.cursor()
    insert_update_sql="""INSERT INTO short_data VALUES(?,?,?,?,?,?,?,?,?)"""
    fromdate='20001201'#공매도 거래가 있던 첫일의 한달 전
    today=datetime.strftime(datetime.today().date(), '%Y%m%d')
    while fromdate < today:
        fromdate=((datetime.strptime(fromdate, '%Y%m%d'))+relativedelta(months=1)).strftime('%Y%m%d')
        df=get_market_short('kospi',None,fromdate)
        for idx, r in df.iterrows():
            cur.execute(insert_update_sql,r)
            
    con.commit()
    con.close()





In [6]:
a = update()